# 取得所有頁面的連結

In [ ]:
def get_links(driver):
    cnt_links = []
#     oldest_page = driver.find_element_by_css_selector("#action-bar-container > div > div.btn-group.btn-group-paging > a:nth-child(1)")
#     oldest_page.click()

    while True:      
        html = driver.page_source
        soup = BeautifulSoup(html,"lxml")
        print("link_a")
        links_boxes = soup.find_all('div', class_='title')
        links = re.findall('<a href="(.*)">', str(links_boxes))
        print("link_b")
        url = "https://www.ptt.cc"
        for link in links:
            if len(link)>0:
                cnt_links.append(url+link)
        print("link_c")
        next_page = driver.find_element_by_css_selector("#action-bar-container > div > div.btn-group.btn-group-paging > a:nth-child(3)")
        #按到最後一頁不能再按, 但還是顯示在網頁上 可用disabled來判斷
        if 'disabled' in next_page.get_attribute('class'):
            break;
        next_page.click()

    return (cnt_links)

# 取得ppt內文

In [ ]:
def inner_cnt(soup_cnt):
    cnt = soup_cnt.select_one('div#main-content')
    selectors_to_del = ['.article-metaline',
                       '.article-metaline-right',
                        'span.f2',
                        'div.push'
                       ]

    for selector in selectors_to_del:
        [tag.extract() for tag in cnt.select(selector)]
    x = cnt.text.replace("\n", "")
    clean_cnt = re.findall('(.*)--', x)
    return(clean_cnt)

# 取得連結內容 

In [ ]:
def get_cnt(client, db, collections, name, driver, cnt_links):
    print('get_cnt_a')
    print(type(cnt_links))
    print("版名: "+ name +" , 連結數"+ str(len(cnt_links)))
    print('get_cnt_b')
    k = 0
    
    for link in cnt_links:
        driver.get(str(link))
        html = driver.page_source

        soup_cnt = BeautifulSoup(html,"lxml")
        print("get_cnt_c")
        ppt_review = {}


        #取得基本資訊
        try:
            metas = soup_cnt.find_all('div', class_ = 'article-metaline')
            ppt_review['title'] = metas[1].text.replace("標題", "")
            ppt_review['time'] = metas[2].text.replace("時間", "")
        except IndexError:
            ppt_review['title'] = ""
            ppt_review['time'] = ""
        print("get_cnt_d")

        #取得評論
        try:
            cmts = soup_cnt.find_all('span', class_ = 'f3 push-content')
            ppt_review['cnt'] = []
            for cmt in cmts:
                x = cmt.text.strip(':')
                ppt_review['cnt'].append(x)
        except Exception as e:
            print (e.__doc__)            
            
        #取得內文
        try:
            ppt_review['article'] = inner_cnt(soup_cnt)
        except Exception as e:
            print (e.__doc__)  
        print("get_cnt_e")
        
        db.ppt_col.insert(ppt_review)

        k+=1
        print("寫入至mongodb第"+ str(k) +"筆 " + name)
    
    return

# 更換不同header

In [ ]:
from fake_useragent import UserAgent
import json

def random_header():
    ua = UserAgent()
    random_header = json.loads(r'''{
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Host": "www.dogforum.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent":"%s"
    }'''%ua.random)
    return random_header

In [ ]:
from bs4 import BeautifulSoup
import time
import re 
from pymongo import MongoClient
import concurrent.futures
import threading
from queue import Queue 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import InvalidElementStateException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
import random
from fake_useragent import UserAgent
import json
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


def getStart():
    #用chrome爬蟲一定要有chromedriver.exe才可以
    chrome_path = r"C:\Users\Java\Desktop\iii_project\textmining\chromedriver.exe"
    
    #把不同的args,亦即header加入webdriver中
    options = webdriver.ChromeOptions()
    args = str(random_header())
    options.add_argument(args)
    options.add_argument('lang=zh_CN.UTF-8')
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=options )
    n = 1
    
    #建立與mongo連線, 並創立db和collection
    client = MongoClient('mongodb://localhost:27017')
    db = client.ppt_db
    collections = db.ppt_col
    
    while not q.empty():
        name = q.get()
        #睡 任意0~10秒讓程式速度降下來, 避免被發現是爬蟲 
        time.sleep(random.randint(0, 10))
        try:
            print(threading.current_thread())
            print("c")
            #開啟google chrome
            driver.get(name)
            
            print(threading.current_thread())
            print("d")
            
            #開始爬內文
            cnt_links = get_links(driver)
            
            print(threading.current_thread())
            print("e")
            
            get_cnt(client, db, collections, name, driver, cnt_links)
            
            print(threading.current_thread())
            print("f")
            
        except(InvalidElementStateException,NoSuchElementException,WebDriverException,Exception) as e:
            print("h")
            fail_store_names.append(name)
            print(threading.current_thread())
            print("error:"+ str(e) + name)
            print(e.with_traceback)
    driver.close() 
    end = time.time()
    
    start_time = time.localtime(start)
    end_time = time.localtime(end)
    start_project= time.strftime('%Y-%m-%d %H:%M:%S', start_time )
    end_project= time.strftime('%Y-%m-%d %H:%M:%S',end_time )
    print(start_project)
    print(end_project)
    print("總共花了: " + str(end - start))
    client.close()

#開啟多個執行緒
def startDriver():
    thread_amount = 4 #開4個thread
    threads = []
    for i in range(thread_amount):  #開thread
        t = threading.Thread(target=getStart)
        #開始爬蟲
        t.start()
        threads.append(t)   
    for thread in threads:
        thread.join()
        


#從這裡開始爬蟲
start = time.time()
fail_store_names = []

#建立queue, 把所有的店名塞進queue裡
q = Queue(maxsize=0)

#ppt健身四大板的連結
ppt_boards = ['https://www.ptt.cc/bbs/FITNESS/index.html', 
             'https://www.ptt.cc/bbs/musclebeach/index.html', 
             'https://www.ptt.cc/bbs/Aerobics/index.html', 
             'https://www.ptt.cc/bbs/BeautyBody/index.html']

#把四大板的連結全部放入queue中
for board in ppt_boards:
    q.put(board)

#開啟執行緒
startDriver()